In [1]:
import os
from tqdm import tqdm
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.sparse import csr_matrix

import warnings

warnings.filterwarnings("ignore")

In [3]:
path = '.ml-latest-small/'
ratings_df = pd.read_csv(path + 'ratings.csv', encoding='utf-8')
movies_df = pd.read_csv(path + 'movies.csv',
                        index_col='movieId',
                        encoding='utf-8')
tags_df = pd.read_csv(path + 'tags.csv', encoding='utf-8')

In [4]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
movies_df.head()

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [6]:
tags_df.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


#### Genres를 이용한 movie representation

In [7]:
total_count = len(movies_df.index)
total_genres = list(
    set([
        genre
        for sublist in list(map(lambda x: x.split('|'), movies_df['genres']))
        for genre in sublist
    ]))

In [8]:
print(f"전체 영화 수: {total_count}")
print(f"장르: {total_genres}")

전체 영화 수: 9742
장르: ['Comedy', 'Children', 'IMAX', 'Musical', 'Drama', 'Horror', 'Thriller', 'Documentary', 'Crime', 'Adventure', 'Romance', 'Fantasy', 'Mystery', 'Western', 'War', 'Sci-Fi', 'Action', 'Film-Noir', '(no genres listed)', 'Animation']


In [9]:
print(len(total_genres))

20


In [14]:
genre_count = dict.fromkeys(total_genres)

for each_genre_list in movies_df['genres']:
    for genre in each_genre_list.split('|'):
        if genre_count[genre] == None:
            genre_count[genre] = 1
        else:
            genre_count[genre] = genre_count[genre]+1

In [15]:
genre_count

{'Comedy': 3756,
 'Children': 664,
 'IMAX': 158,
 'Musical': 334,
 'Drama': 4361,
 'Horror': 978,
 'Thriller': 1894,
 'Documentary': 440,
 'Crime': 1199,
 'Adventure': 1263,
 'Romance': 1596,
 'Fantasy': 779,
 'Mystery': 573,
 'Western': 167,
 'War': 382,
 'Sci-Fi': 980,
 'Action': 1828,
 'Film-Noir': 87,
 '(no genres listed)': 34,
 'Animation': 611}

In [16]:
# 장르별 가중치
for each_genre in genre_count:
    genre_count[each_genre] = np.log10(total_count / genre_count[each_genre])

genre_count

{'Comedy': 0.4139225416416778,
 'Children': 1.1664800458677336,
 'IMAX': 1.7899910382813284,
 'Musical': 1.4649016584241867,
 'Drama': 0.3490620385623247,
 'Horror': 0.9983092704481497,
 'Thriller': 0.7112681505684965,
 'Documentary': 1.3451954487495636,
 'Crime': 0.9098289421369025,
 'Adventure': 0.8872447746804204,
 'Romance': 0.7856152382210405,
 'Fantasy': 1.0971106675631865,
 'Mystery': 1.2304935032683613,
 'Western': 1.7659316540881678,
 'War': 1.4065847623240424,
 'Sci-Fi': 0.9974220495432563,
 'Action': 0.7266719338379385,
 'Film-Noir': 2.0491288726171324,
 '(no genres listed)': 2.457169208193496,
 'Animation': 1.2026069149931968}

In [19]:
# create genre representations
genre_representation = pd.DataFrame(columns=sorted(total_genres),
                                    index=movies_df.index)
for index, each_row in tqdm(movies_df.iterrows()):
    dict_temp = {i: genre_count[i] for i in each_row['genres'].split('|')}
    row_to_add = pd.DataFrame(dict_temp, index=[index])
    genre_representation.update(row_to_add)

9742it [00:29, 330.34it/s]


In [20]:
genre_representation.sample(5)

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,
5847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.349062,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2887,NaN,0.726672,NaN,NaN,NaN,0.413923,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.998309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83827,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.345195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4928,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.349062,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Tag를 이용한 Movie Representation

In [21]:
tags_df.head(5)

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [22]:
movies_df.loc[89774]

title     Warrior (2011)
genres             Drama
Name: 89774, dtype: object

In [23]:
# get unique tag
tag_column = list(map(lambda x: x.split(','), tags_df['tag']))
unique_tags = list(
    set(
        list(
            map(lambda x: x.strip(),
                list([tag for sublist in tag_column for tag in sublist])))))

print(unique_tags)

['ships', 'off-beat comedy', 'lyrical', 'vampires', 'Uma Thurman', 'assassin-in-training (scene)', 'Wizards', 'mind-bending', 'pixar', 'entirely dialogue', 'John Cusack', 'thought-provoking', 'leopard', 'noir', 'Jim Carrey', 'courtroom drama', 'indiana jones', 'Disaster', 'thought provoking', 'spacecraft', 'big top', 'sisters', 'hotel', 'lesbian', 'plastic surgery', 'double life', 'Dodie Smith', 'crappy sequel', 'Stand Up', 'new york', 'mafia', 'Christopher Lloyd', 'Hayao Miyazaki', 'Pixar', 'New York City', 'Will Ferrell', 'Well Done', 'tragedy', 'blind', 'Jason Biggs', 'mystery', 'big name actors', 'anti-Semitism', 'funny', 'Not available from Netflix', 'Angelina Jolie', "Eugene O'Neill", 'drugs', 'chess', 'terminal illness', 'Michigan', 'small time criminals', 'video', 'Seann William Scott', 'modern fantasy', 'violence', 'TOGA', 'suspense', 'dating', 'James Fennimore Cooper', 'classic sci-fi', 'hugh jackman', 'predictable', 'absorbing', 'survival', 'journalism', 'shakespeare', 'not 

In [24]:
print(len(tag_column))
print(len(unique_tags))

3683
1589


In [29]:
# Compute IDF for tag
total_movie_count = len(set(tags_df['movieId']))
# key: tag, value: number of movies with such tag
tag_count_dict = dict.fromkeys(unique_tags)

for each_movie_tag_list in tags_df['tag']:
    for tag in each_movie_tag_list.split(","):
        if tag_count_dict[tag.strip()] == None:
            tag_count_dict[tag.strip()] = 1
        else:
            tag_count_dict[tag.strip()] += 1

tag_idf = dict()
for each_tag in tag_count_dict:
    tag_idf[each_tag] = np.log10(total_movie_count / tag_count_dict[each_tag])

tag_idf

{'ships': 3.196452541703389,
 'off-beat comedy': 2.7193312869837265,
 'lyrical': 2.895422546039408,
 'vampires': 2.895422546039408,
 'Uma Thurman': 3.196452541703389,
 'assassin-in-training (scene)': 3.196452541703389,
 'Wizards': 2.5943925503754266,
 'mind-bending': 3.196452541703389,
 'pixar': 2.895422546039408,
 'entirely dialogue': 3.196452541703389,
 'John Cusack': 3.196452541703389,
 'thought-provoking': 1.816241299991783,
 'leopard': 3.196452541703389,
 'noir': 3.196452541703389,
 'Jim Carrey': 2.895422546039408,
 'courtroom drama': 2.895422546039408,
 'indiana jones': 2.895422546039408,
 'Disaster': 3.196452541703389,
 'thought provoking': 3.196452541703389,
 'spacecraft': 3.196452541703389,
 'big top': 3.196452541703389,
 'sisters': 3.196452541703389,
 'hotel': 3.196452541703389,
 'lesbian': 3.196452541703389,
 'plastic surgery': 3.196452541703389,
 'double life': 3.196452541703389,
 'Dodie Smith': 3.196452541703389,
 'crappy sequel': 3.196452541703389,
 'Stand Up': 3.19645254

In [26]:
len(tag_idf.keys())

1589

In [50]:
# movie representations
tag_representation = pd.DataFrame(columns=sorted(unique_tags),
                                  index=list(set(tags_df['movieId'])))
for name, group in tqdm(tags_df.groupby(by='movieId')):
    temp_list = list(map(lambda x: x.split(','), list(group['tag'])))
    temp_tag_list = list(
        set(
            list(
                map(lambda x: x.strip(),
                    list([tag for sublist in temp_list for tag in sublist])))))

    dict_temp = {i: tag_idf[i.strip()] for i in temp_tag_list}
    row_to_add = pd.DataFrame(dict_temp, index=[group['movieId'].values[0]])
    tag_representation.update(row_to_add)

tag_representation = tag_representation.sort_index(0)

100%|██████████████████████████████████████████████| 1572/1572 [02:10<00:00, 12.09it/s]


In [31]:
tag_representation.sample(5)

,"""artsy""",06 Oscar Nominated Best Movie - Animation,1900s,1920s,1950s,1960s,1970s,1980s,1990s,2001-like,...,women,wonderwoman,workplace,writing,wrongful imprisonment,wry,younger men,zither,zoe kazan,zombies
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
movies_df.head()

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [33]:
tag_representation.loc[1].dropna()

fun      2.497483
pixar    2.895423
Name: 1, dtype: object

In [34]:
tag_representation.loc[2].dropna()

Robin Williams      2.719331
fantasy             2.418301
game                3.196453
magic board game    3.196453
Name: 2, dtype: object

In [35]:
print(genre_representation.shape)
print(tag_representation.shape)

(9742, 20)
(1572, 1589)


## Final Movie Represenation
- genre와 tag로 만들어진 representation을 합쳐서 각 movie의 vector로 만든다

In [36]:
movie_representation = pd.concat([genre_representation, tag_representation],
                                 axis=1).fillna(0)
print(movie_representation.shape)
print(movie_representation.describe())

(9742, 1609)
       (no genres listed)       Action    Adventure    Animation     Children  \
count         9742.000000  9742.000000  9742.000000  9742.000000  9742.000000   
mean             0.008576     0.136354     0.115027     0.075425     0.079506   
std              0.144915     0.283726     0.298052     0.291593     0.293989   
min              0.000000     0.000000     0.000000     0.000000     0.000000   
25%              0.000000     0.000000     0.000000     0.000000     0.000000   
50%              0.000000     0.000000     0.000000     0.000000     0.000000   
75%              0.000000     0.000000     0.000000     0.000000     0.000000   
max              2.457169     0.726672     0.887245     1.202607     1.166480   

            Comedy        Crime  Documentary        Drama      Fantasy  ...  \
count  9742.000000  9742.000000  9742.000000  9742.000000  9742.000000  ...   
mean      0.159587     0.111978     0.060756     0.156257     0.087728  ...   
std       0.201476  

## Contents 유사도 평가
- Cosine similarity를 사용

In [37]:
from sklearn.metrics.pairwise import cosine_similarity


def cos_sim_matrix(a, b):
    cos_sim = cosine_similarity(a, b)
    result_df = pd.DataFrame(data=cos_sim, index=[a.index])

    return result_df

In [38]:
print(movie_representation.head())

   (no genres listed)  Action  Adventure  Animation  Children    Comedy  \
1                 0.0     0.0   0.887245   1.202607   1.16648  0.413923   
2                 0.0     0.0   0.887245   0.000000   1.16648  0.000000   
3                 0.0     0.0   0.000000   0.000000   0.00000  0.413923   
4                 0.0     0.0   0.000000   0.000000   0.00000  0.413923   
5                 0.0     0.0   0.000000   0.000000   0.00000  0.413923   

   Crime  Documentary     Drama   Fantasy  ...  women  wonderwoman  workplace  \
1    0.0          0.0  0.000000  1.097111  ...    0.0          0.0        0.0   
2    0.0          0.0  0.000000  1.097111  ...    0.0          0.0        0.0   
3    0.0          0.0  0.000000  0.000000  ...    0.0          0.0        0.0   
4    0.0          0.0  0.349062  0.000000  ...    0.0          0.0        0.0   
5    0.0          0.0  0.000000  0.000000  ...    0.0          0.0        0.0   

   writing  wrongful imprisonment  wry  younger men  zither  z

In [39]:
cs_df = cos_sim_matrix(movie_representation, movie_representation)
cs_df.head()

,0,1,2,3,4,5,6,7,8,9,...,9732,9733,9734,9735,9736,9737,9738,9739,9740,9741
1,1.000000,0.124438,0.008403,0.040571,0.011755,0.0,0.016339,0.331122,0.0,0.131794,...,0.064466,0.260941,0.071492,0.27171,0.0,0.348295,0.379492,0.000000,0.232553,0.093519
2,0.124438,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.240843,0.0,0.095861,...,0.000000,0.000000,0.000000,0.00000,0.0,0.108082,0.117763,0.000000,0.000000,0.000000
3,0.008403,0.000000,1.000000,0.179391,0.011294,0.0,0.072246,0.000000,0.0,0.000000,...,0.006560,0.000000,0.068686,0.00000,0.0,0.020322,0.022142,0.000000,0.000000,0.089849
4,0.040571,0.000000,0.179391,1.000000,0.054530,0.0,0.348828,0.000000,0.0,0.000000,...,0.031674,0.101979,0.567487,0.00000,0.0,0.098119,0.106908,0.365843,0.000000,0.433821
5,0.011755,0.000000,0.011294,0.054530,1.000000,0.0,0.640342,0.000000,0.0,0.000000,...,0.009177,0.000000,0.096091,0.00000,0.0,0.028429,0.030976,0.000000,0.000000,0.125697


In [40]:
cs_df.shape

(9742, 9742)

In [41]:
print(cs_df.shape)
print(cs_df[1].sort_values(ascending=False))

(9742, 9742)
2         1.000000
46972     0.322201
158813    0.300850
119655    0.300850
80748     0.300850
            ...   
4921      0.000000
4920      0.000000
4919      0.000000
4917      0.000000
193609    0.000000
Name: 1, Length: 9742, dtype: float64


In [42]:
print(movies_df.loc[1])
print(movies_df.loc[46972])
print(movies_df.loc[126142])
print(movies_df.loc[2043])
print(movies_df.loc[2399])

title                                Toy Story (1995)
genres    Adventure|Animation|Children|Comedy|Fantasy
Name: 1, dtype: object
title     Night at the Museum (2006)
genres    Action|Comedy|Fantasy|IMAX
Name: 46972, dtype: object
title     The Cave of the Golden Rose (1991)
genres            Adventure|Children|Fantasy
Name: 126142, dtype: object
title     Darby O'Gill and the Little People (1959)
genres                   Adventure|Children|Fantasy
Name: 2043, dtype: object
title     Santa Claus: The Movie (1985)
genres       Adventure|Children|Fantasy
Name: 2399, dtype: object


## 추천시스템의 성능 평가
- 학습셋과 테스트셋을 나눈다.
- 테스트셋에서 예측한 평점과 실제 평점의 RMSE를 구한다.


In [43]:
train_df, test_df = train_test_split(ratings_df,
                                     test_size=0.2,
                                     random_state=123)

In [44]:
print(train_df.shape)
print(test_df.shape)

(80668, 4)
(20168, 4)


In [49]:
test_userids = list(set(test_df.userId.values))
test_userids[:5]

[1, 2, 3, 4, 5]

In [46]:
result_df = pd.DataFrame()

for user_id in tqdm(test_userids):
    user_record_df = train_df.loc[train_df.userId == int(user_id), :]

    user_sim_df = cs_df.loc[
        user_record_df['movieId']]  # (n, 9742); n은 userId가 평점을 매긴 영화 수
    user_rating_df = user_record_df[['rating']]  # (n, 1)
    sim_sum = np.sum(user_sim_df.T.to_numpy(), -1)  # (9742, 1)
    # print("user_id=", i, user_record_df.shape, user_sim_df.T.shape, user_rating_df.shape, sim_sum.shape)

    prediction = np.matmul(user_sim_df.T.to_numpy(),
                           user_rating_df.to_numpy()).flatten() / (
                               sim_sum + 1)  # (9742, 1)

    prediction_df = pd.DataFrame(prediction, index=cs_df.index).reset_index()
    prediction_df.columns = ['movieId', 'pred_rating']
    prediction_df = prediction_df[[
        'movieId', 'pred_rating'
    ]][prediction_df.movieId.isin(
        test_df[test_df.userId == user_id]['movieId'].values)]

    temp_df = prediction_df.merge(test_df[test_df.userId == user_id],
                                  on='movieId')
    result_df = pd.concat([result_df, temp_df], axis=0)

100%|███████████████████████████████████████████████| 610/610 [00:05<00:00, 108.86it/s]


In [47]:
result_df.head(10)

,movieId,pred_rating,userId,rating,timestamp
0,6,4.136308,1,4.0,964982224
1,151,4.261184,1,5.0,964984041
2,231,4.232403,1,5.0,964981179
3,423,4.088333,1,3.0,964982363
4,441,4.058008,1,4.0,964980868
5,543,3.688726,1,4.0,964981179
6,804,4.063459,1,4.0,964980499
7,954,2.300271,1,5.0,964983219
8,1029,4.531640,1,5.0,964982855
9,1030,4.482305,1,3.0,964982903


In [48]:
mse = mean_squared_error(y_true=result_df['rating'].values,
                         y_pred=result_df['pred_rating'].values)
rmse = np.sqrt(mse)

print(mse, rmse)

1.3970670771596811 1.181975920719065
